# Humidity Data Processing
This notebook loads raw humidity data, handles missing values, cleans timestamp formats, and exports a processed dataset ready for analysis.

In [40]:
# Load libraries
import pandas as pd
import glob
import os

RAW_DEMAND_DIR = "data/raw/demand"
RAW_WEATHER_DIR = "data/raw/weather"
PROCESSED_DIR = "data/processed"

STATES = ["VIC", "NSW", "QLD", "SA", "TAS"]

# Map REGION codes in AEMO data to clean state names
REGION_TO_STATE = {
    "NSW1": "NSW",
    "QLD1": "QLD",
    "SA1":  "SA",
    "TAS1": "TAS",
    "VIC1": "VIC",
}

WEATHER_FILES = {
    "VIC": ["HM01X_Data_086338_999999999743964.txt","HM01X_Data_086338_999999999743964.txt"],
    "NSW": ["HM01X_Data_066062_999999999743964.txt"],
    "QLD": ["HM01X_Data_040913_999999999743964.txt"],
    "SA":  ["HM01X_Data_023090_999999999743964.txt"],
    "TAS": ["HM01X_Data_094029_999999999743964.txt"],
}

# Rename columns & variables
WEATHER_DATETIME_COLS = {
    "year":   "Year Month Day Hour Minutes in YYYY",
    "month":  "MM",
    "day":    "DD",
    "hour":   "HH24",
    "minute": "MI format in Local time",
}

HUMIDITY_RAW_COL = "Relative humidity in percentage %"
TEMP_RAW_COL     = "Air Temperature in degrees C"

HUMIDITY_COL = "Humidity (%)"
TEMP_COL     = "Temperature"

## Create Functions to Load Files

In [30]:
def load_demand_for_state(state: str) -> pd.DataFrame | None:
    """
    Read and combine all raw demand CSVs for a single state.
    Renames columns and parses Datetime.
    """
    pattern = os.path.join(RAW_DEMAND_DIR, f"*{state}*.csv")
    demand_files = sorted(glob.glob(pattern))

    if not demand_files:
        print(f"[WARN] No demand files found for {state} (pattern: {pattern})")
        return None

    df_list = [pd.read_csv(f) for f in demand_files]
    demand_df = pd.concat(df_list, ignore_index=True)

    demand_df = demand_df.rename(
        columns={
            "SETTLEMENTDATE": "Datetime",
            "TOTALDEMAND": "Total Demand",
            "REGION": "State",
        }
    )
    demand_df = demand_df.drop(columns=["PERIODTYPE"], errors="ignore")

    # Parse datetime
    demand_df["Datetime"] = pd.to_datetime(demand_df["Datetime"], format="mixed", utc=True)

    # Map REGION codes
    demand_df["State"] = demand_df["State"].replace(REGION_TO_STATE)

    # Drop timezone for merging with local BOM timestamps
    demand_df["Datetime"] = demand_df["Datetime"].dt.tz_localize(None)

    print(f"[INFO] Demand for {state}: {demand_df.shape}")
    return demand_df


In [38]:
def load_weather_for_state(state: str) -> pd.DataFrame | None:
    """
    Read and combine BOM weather files for a single state,
    construct Datetime, and keep only humidity + temperature.
    """
    filenames = WEATHER_FILES.get(state, [])
    if not filenames:
        print(f"[WARN] No weather files configured for {state} in WEATHER_FILES.")
        return None

    dfs = []
    for fname in filenames:
        path = os.path.join(RAW_WEATHER_DIR, fname)
        if not os.path.exists(path):
            print(f"[WARN] Weather file not found: {path}")
            continue
        dfs.append(pd.read_csv(path))

    if not dfs:
        print(f"[WARN] No weather data actually loaded for {state}.")
        return None

    weather_df = pd.concat(dfs, ignore_index=True)

    # Build Datetime from BOM columns
    weather_df["Datetime"] = pd.to_datetime(
        {
            "year":   weather_df[WEATHER_DATETIME_COLS["year"]],
            "month":  weather_df[WEATHER_DATETIME_COLS["month"]],
            "day":    weather_df[WEATHER_DATETIME_COLS["day"]],
            "hour":   weather_df[WEATHER_DATETIME_COLS["hour"]],
            "minute": weather_df[WEATHER_DATETIME_COLS["minute"]],
        },
        errors="coerce"
    )

    # Drop timezone to match demand_df
    weather_df["Datetime"] = weather_df["Datetime"].dt.tz_localize(None)

    # Check required columns exist
    missing_cols = [c for c in [HUMIDITY_RAW_COL, TEMP_RAW_COL] if c not in weather_df.columns]
    if missing_cols:
        print(f"[WARN] Missing columns for {state}: {missing_cols}")
        print("Available columns:", list(weather_df.columns)[:15], "...")
        return None

    # Rename raw cols -> clean cols
    weather_df = weather_df.rename(
        columns={
            HUMIDITY_RAW_COL: HUMIDITY_COL,
            TEMP_RAW_COL: TEMP_COL,
        }
    )

    # Keep only Datetime + humidity + temperature
    weather_df = weather_df[["Datetime", HUMIDITY_COL, TEMP_COL]]

    # Convert to numeric
    weather_df[HUMIDITY_COL] = pd.to_numeric(weather_df[HUMIDITY_COL], errors="coerce")
    weather_df[TEMP_COL] = pd.to_numeric(weather_df[TEMP_COL], errors="coerce")

    # Sort by Datetime
    weather_df = weather_df.sort_values("Datetime")

    print(f"[INFO] Weather (Humidity+Temp) for {state}: {weather_df.shape}")
    return weather_df

## Create Function to Merge and Save Files

In [34]:
def process_state(state: str) -> pd.DataFrame | None:
    """
    Full pipeline for one state:
    - load demand
    - load humidity (only)
    - inner merge on Datetime
    - save to data/processed/{STATE}_clean.csv
    """
    demand_df = load_demand_for_state(state)
    humidity_df = load_weather_for_state(state)

    if demand_df is None or humidity_df is None:
        print(f"[WARN] Skipping {state} because demand or humidity is missing.")
        return None

    merged_df = pd.merge(
        demand_df,
        humidity_df,
        on="Datetime",
        how="inner",
    )

    out_path = os.path.join(PROCESSED_DIR, f"{state}_clean.csv")
    merged_df.to_csv(out_path, index=False)

    print(f"[OK] {state}: saved {out_path}, shape={merged_df.shape}")
    return merged_df

## Clean Files & Build Master File

In [41]:
all_states_dfs = []

for state in STATES:
    df_state = process_state(state)
    if df_state is not None:
        # Ensure State column exists
        if "State" not in df_state.columns:
            df_state["State"] = state
        all_states_dfs.append(df_state)

if all_states_dfs:
    master = pd.concat(all_states_dfs, ignore_index=True)
    master_path = os.path.join(PROCESSED_DIR, "master_data.csv")
    master.to_csv(master_path, index=False)
    print("[OK] Master file created:", master_path, master.shape)
else:
    print("[WARN] No states processed; master file not created.")

[INFO] Demand for VIC: (350632, 4)


/var/folders/0d/qf_dfhwn3vjd4fvry35670240000gn/T/ipykernel_85738/1358033223.py:17: DtypeWarning: Columns (12,16,22,24,26,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(path))
/var/folders/0d/qf_dfhwn3vjd4fvry35670240000gn/T/ipykernel_85738/1358033223.py:17: DtypeWarning: Columns (12,16,22,24,26,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(path))


[INFO] Weather (Humidity+Temp) for VIC: (238886, 3)
[OK] VIC: saved data/processed/VIC_clean.csv, shape=(230880, 6)
[INFO] Demand for NSW: (350632, 4)


/var/folders/0d/qf_dfhwn3vjd4fvry35670240000gn/T/ipykernel_85738/1358033223.py:17: DtypeWarning: Columns (12,14,18,20,28,32) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(path))


[INFO] Weather (Humidity+Temp) for NSW: (350945, 3)
[OK] NSW: saved data/processed/NSW_clean.csv, shape=(349699, 6)
[INFO] Demand for QLD: (350632, 4)


...

[INFO] Weather (Humidity+Temp) for QLD: (350977, 3)
[OK] QLD: saved data/processed/QLD_clean.csv, shape=(349294, 6)
[INFO] Demand for SA: (350632, 4)


/var/folders/0d/qf_dfhwn3vjd4fvry35670240000gn/T/ipykernel_85738/1358033223.py:17: DtypeWarning: Columns (12,14,18,20,22,24,26,28,32) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(path))


[INFO] Weather (Humidity+Temp) for SA: (355906, 3)
[OK] SA: saved data/processed/SA_clean.csv, shape=(349176, 6)
[INFO] Demand for TAS: (256437, 4)


/var/folders/0d/qf_dfhwn3vjd4fvry35670240000gn/T/ipykernel_85738/1358033223.py:17: DtypeWarning: Columns (12,14,18,20,22,24,26,28,32) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(path))


[INFO] Weather (Humidity+Temp) for TAS: (386362, 3)
[OK] TAS: saved data/processed/TAS_clean.csv, shape=(254964, 6)
[OK] Master file created: data/processed/master_data.csv (1534013, 6)
